In [ ]:
import string
import json
import copy
from tqdm import tqdm
import re
import pandas as pd

In [ ]:
train_df = pd.read_csv("data/salary_labelled_development_set_cleaned.csv")
test_df = pd.read_csv('data/salary_labelled_test_set_cleaned.csv')

In [ ]:
train_df

,job_id,job_title,job_ad_details,nation_short_desc,salary_additional_text,y_true
0,72000415,Financial Account - Call Center Agent - Up to 34k,\n \n Job Opening \n \n \n Financial Account -...,PH,NaN,17500-17500-PHP-MONTHLY
1,69481519,Aspiring Call Center Agents - Work from Home -...,\n\nJob Opening\n\nAspiring Call Center Agents...,PH,NaN,16000-16000-PHP-MONTHLY
2,55838599,Production Staff Required - Afternoon & Night-...,Original Foods Baking Co. is one of New Zealan...,NZ,NaN,0-0-None-None
3,64369104,Payer Analyst,The Payer Analyst individual is assigned to ...,PH,-,0-0-None-None
4,54861511,"Solicitor, Restructuring",The DLA Piper team operates across more than 4...,AUS,NaN,0-0-None-None
...,...,...,...,...,...,...
2248,75640730,"Senior Manager, Process Excellence and Transfo...",Our client is a well-established consumer good...,HK,NaN,0-0-None-None
2249,77931852,Accounting Manager - Hotel Business,Position: Accounting ManagerJob DescriptionCo...,TH,NaN,0-0-None-None
2250,54359262,Technical Operator - Audio Visual,Your mission: To create memorable experiences ...,AUS,NaN,0-0-None-None
2251,73457326,Customer Service Officer,Working Day: 5-6-5-6 work weeks (rotating base...,SG,"$2,500 – $3,000 per month",2500-3000-SGD-MONTHLY


# Loading Qwen

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-0.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
def call_model(messages):
  text = tokenizer.apply_chat_template(
      messages,
      tokenize=False,
      add_generation_prompt=True
  )
  model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

  generated_ids = model.generate(
      **model_inputs,
      max_new_tokens=512
  )
  generated_ids = [
      output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
  ]

  response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
  return response

In [ ]:
#== step 1: role explanation
def step_1():
  messages = [
      {"role": "system", "content": "You are an expert job ad annotator. Your task is to extract structured salary information from job descriptions in the format: min-max-currency-frequency. If salary is not found, return: 0-0-None-None."},
      {"role": "user", "content": "Based on your role, can you briefly explain what constitutes a salary range, currency and frequency in a job listing?"}
  ]

  response = call_model(messages)
  messages.append({"role": "assistant", "content": response[0]})
  return response[0], messages


### RB

In [ ]:
#== few-shot examples
def fewshot(messages):
  few_shot_indices = [1030, 1041, 1025, 1022, 1011]

  for i in few_shot_indices:
    desc = {
      "job_title": train_df.iloc[i].job_title,
      "job_ad_details": train_df.iloc[i].job_ad_details,
      "nation_short_desc": train_df.iloc[i].nation_short_desc,
      "salary_additional_text": train_df.iloc[i].salary_additional_text,
    }

    desc_str = str(desc)
     # add the description
    messages.append({
      "role": "user",
      "content": desc_str
    })

    label = train_df.iloc[i].y_true
    label_str = str(label)

    # add the output
    messages.append({
        "role": "assistant",
        "content": label_str
    })

  return messages


In [ ]:
fewshot([])

[{'role': 'user',
  'content': '{\'job_title\': \'Thailand based Online English Teacher  Expat\', \'job_ad_details\': "\\xa0JOIN OUR TEAM OF INSPIRING EDUCATORS!\\xa0Hello Passionate Educators!Are you a truly exceptional teacher, fueled by the desire to ignite a passion for learning in those you teach? The Really Great Teacher Company invites you to embark on a rewarding journey of personal and professional growth while making a positive impact on students worldwide through the art of teaching English. WHAT AWAITS YOU:\\xa0CAREER ADVANCEMENT:\\xa0Countless opportunities await your upward trajectory into more senior roles.\\xa0DEDICATED SUPPORT TEAM:\\xa0We\'ve got your back with all the materials and human support you need to shine.\\xa0PROFESSIONAL DEVELOPMENT:\\xa0Our Inspire Academy offers a treasure trove of free courses, allowing you to chart your professional growth at your own pace.\\xa0PERSONAL DEVELOPMENT:\\xa0Join our Evolve program and revel in wellness, fitness, fun, and co

In [ ]:
#== step 2: setting sub-task --> ask for salary patterns
def step_2(messages):
  messages.append({
      "role": "user",
      "content": "As a salary extractor, what are some common phrases or numeric patterns that indicate a salary range, currency and frequency in a job description?"
  })

  response = call_model(messages)
  messages.append({"role": "assistant", "content": response[0]})
  return response, messages

In [ ]:
step_2(step_1()[1])

(['Common Phrases:\n1. "Range of [min salary] to [max salary]"\n2. "Salary range [min salary] to [max salary]"\n3. "Salary within [min salary] to [max salary]"\n4. "Salary range from [min salary] to [max salary]"\n5. "Salary bracket between [min salary] and [max salary]"\n6. "Salary scale [min salary] to [max salary]"\n7. "Salary range [min salary] to [max salary]"\n8. "Salary band from [min salary] to [max salary]"\n9. "Salary range [min salary] to [max salary]"\n10. "Salary scale between [min salary] and [max salary]"\n11. "Salary bracket between [min salary] and [max salary]"\n12. "Salary scale [min salary] to [max salary]"\n13. "Salary range [min salary] to [max salary]"\n14. "Salary band from [min salary] to [max salary]"\n15. "Salary range [min salary] to [max salary]"\n16. "Salary scale [min salary] to [max salary]"\n17. "Salary bracket between [min salary] and [max salary]"\n18. "Salary scale [min salary] to [max salary]"\n19. "Salary range [min salary] to [max salary]"\n20. "S

### IC

In [ ]:
#== step 3: ask if salary info is in the description
def step_3(messages_static, desc_str):
  messages = copy.deepcopy(messages_static)

  messages.append({
      "role": "user",
      "content": f"{desc_str} does this job description include any salary-related information? Just respond with 'Yes' or 'No'."
  })

  response = call_model(messages)
  messages.append({"role": "assistant", "content": response[0]})

  p3_content = response[0]

  # clean and check the response
  response_p3 = p3_content.strip().lower()
  response_p3 = response_p3.translate(str.maketrans('', '', string.punctuation))
  print(f"step3 messages: {len(messages)}")
  return response_p3, messages

In [ ]:
def step_4(response_p3, messages):
  if response_p3 == "yes":
      messages.append({
          "role": "user",
          "content": "Extract the salary range or salary-related phrases from the text verbatim. Respond in JSON: {\"Clue\": \"\"}."
      })
  elif response_p3 == "no":
      messages.append({
          "role": "user",
          "content": "Briefly explain why there is no salary information (e.g., 'No mention of pay or compensation'). Respond in JSON: {\"Clue\": \"\"}."
      })
  else:
      raise Exception(f"Unexpected model output: {response_p3}")

  response = call_model(messages)
  messages.append({"role": "assistant", "content": response[0]})
  print(f"step4 messages: {len(messages)}")

  return response[0], messages

In [ ]:

#== step 5: use the clue to generate the final output
def step_5(messages):
  messages.append({
      "role": "user",
      "content": (
          "Based on the extracted salary clue, return a structured salary in JSON. "
          "Use 'nation_short_desc' to determine the correct currency. "
          "Use this format: {\"MinSalary\": \"\", \"MaxSalary\": \"\", \"Currency\": \"\", \"Frequency\": \"\"}. "
          "If not provided explicitly, output 0 for \"MinSalary\" and \"MaxSalary\", and \"None\" for \"Currency\" and \"Frequency\". "
          "If the salary is mentioned, always output a range, where MinSalary and MaxSalary can be equal. "
          "Output the currency as 3 letters. Use adverb to output frequency (yearly, monthly, daily or hourly)."
      )
  })


  response = call_model(messages)

  answer_str = response[0]

  print(f"step5 messages: {len(messages)}")

  # format and print the output
  try:
      answer = json.loads(answer_str)
      label = f"{answer['MinSalary']}-{answer['MaxSalary']}-{answer['Currency'].upper()}-{answer['Frequency'].upper()}"

  except json.JSONDecodeError:
      print("Failed to parse model output as JSON:")
      label = "ERROR " + answer_str

  return label, answer_str

In [ ]:
def RBIC_static_messages(verbose=False, add_fewshot=True):
  response, messages = step_1()
  if verbose: print(f"RB step 1: {response}\n")

  response, messages = step_2(messages)
  if verbose: print(f"RB step 2: {response}\n")

  if add_fewshot:
    messages = fewshot(messages)
    if verbose: print(f"Fewshot examples added\n")
  return messages


def RBIC(messages, desc_str, verbose=False):
  response_p3, messages_local = step_3(messages, desc_str)
  if verbose: print(f"IC step 1: {response_p3}\n")

  response, messages_local = step_4(response_p3, messages_local)
  if verbose: print(f"IC step 2: {response}\n")

  label, answer_str = step_5(messages_local)
  if verbose: print(f"IC step 3 (Final): {label}\n")
  if verbose: print(f"IC step 3 (Final Raw): {answer_str}\n")

  return label


# RBIC + Few shot

# Qualitative Tests

In [ ]:
messages_static = RBIC_static_messages(verbose=True, add_fewshot=True)

RB step 1: A salary range typically refers to the upper limit and lower limit of pay for a specific position within an organization or company. Currency frequency indicates how often the salary is paid, such as monthly, quarterly, annually, etc.

For example:
- **Salary Range**: $50k - $70k (monthly)
- **Currency Frequency**: Monthly

If there's no mention of these details, it might indicate that the salary is not disclosed or is confidential.

RB step 2: ['Common phrases or numeric patterns indicating a salary range, currency, and frequency include:\n\n- **$X-Y** where X is the minimum and Y is the maximum salary.\n- **M/M/Y** where M stands for monthly, M stands for half-monthly, and Y stands for yearly.\n- **Y/Y** where Y represents years and Y/Y means annualized.\n- **MM/YR** where MM represents months and YR represents years.\n\nThese terms help in understanding the scope and scale of the salary range mentioned in the job description.']

Fewshot examples added



In [ ]:
ind = 345

desc = {
    "job_title": train_df.iloc[ind].job_title,
    "job_ad_details": train_df.iloc[ind].job_ad_details,
    "nation_short_desc": train_df.iloc[ind].nation_short_desc,
    "salary_additional_text": train_df.iloc[ind].salary_additional_text,
  }
desc_str = str(desc)

label = train_df.iloc[ind].y_true
label_str = str(label)

print(f"len of messages_static {len(messages_static)}")
label_pred = RBIC(messages_static, desc_str, verbose=True)


print(f"pred = {label_pred}")
print(f"truth = {label_str}")

len of messages_static 15
step3 messages: 17
IC step 1: yes

step4 messages: 19
IC step 2: {"Clue": "Salary range: $70,049 - $98,351"}

step5 messages: 20
IC step 3 (Final): $70-$98-USD-MONTHLY

IC step 3 (Final Raw): {"MinSalary": "$70", "MaxSalary": "$98", "Currency": "USD", "Frequency": "monthly"}

pred = $70-$98-USD-MONTHLY
truth = 70049-98351-AUD-ANNUAL


## Quantitative Tests

In [ ]:
test_pred_df = pd.DataFrame(columns=["y_pred"])
test_pred_df

,y_pred


In [ ]:
for i in tqdm(range(len(test_df))):
    desc = {
        "job_title": test_df.iloc[i].job_title,
        "job_ad_details": test_df.iloc[i].job_ad_details,
        "nation_short_desc": test_df.iloc[i].nation_short_desc,
        "salary_additional_text": test_df.iloc[i].salary_additional_text,
    }
    desc_str = str(desc)

    label = test_df.iloc[i].y_true
    label_str = str(label)

    label_pred = RBIC(messages_static, desc_str)

    test_pred_df.loc[len(test_pred_df)] = label_pred

# export the dataframe to a new csv file
test_pred_df.to_csv('salary_labelled_test_set_qwen_rbic_fewshot_preds.csv', index=False)

## **Post processing few shot**
fix error json outputs from model

In [ ]:
rbic_df = pd.read_csv("salary_labelled_test_set_qwen_rbic_fewshot_preds.csv")
rbic_df

,y_pred
0,"ERROR ```json\n{\n ""MinSalary"": """",\n ""MaxSa..."
1,$1000-$2000-USD-MONTHLY
2,$3000-$5000-USD-YEARLY
3,"$11,000-$11,000-USD-MONTHLY"
4,"$2,000-$3,000-USD-MONTHLY"
...,...
571,$26.24-$26.24-THB-YEARLY
572,$2500-$3500-USD-MONTHLY
573,27.36-28.36-AUD-MONTHLY
574,$1500.00-$2500.00-USD-MONTHLY


In [ ]:
post_process_fewshot_df = pd.DataFrame(columns=["y_pred"])

In [ ]:
# include missing quote for value of minSalary
cleaned_text = rbic_df.iloc[120].y_pred
cleaned_text = re.sub(r'(฿[\d]+)', r'"\1"', cleaned_text)
rbic_df.iloc[120].y_pred = cleaned_text

In [ ]:
for i in rbic_df.index:
  prediction = rbic_df.iloc[i].y_pred
  if "ERROR" in rbic_df.iloc[i].y_pred :
    cleaned_text = prediction.replace("ERROR", "").replace("```json", "").replace("```", "").strip()
    # remove any "}}" at the end
    cleaned_text = cleaned_text.replace("}}", "}")
    try:
      loaded = json.loads(cleaned_text)
      label = f"{loaded['MinSalary']}-{loaded['MaxSalary']}-{loaded['Currency'].upper()}-{loaded['Frequency'].upper()}"
      post_process_fewshot_df.loc[i] = label
    except:
      print("failed didnt work", cleaned_text, "at index", i)

post_process_fewshot_df.to_csv('2+post_processed_salary_labelled_test_set_qwen_rbic_fewshot_preds.csv', index=False)


In [ ]:
post_process_fewshot_df['y_pred']

,y_pred


# **No Few Shot**


In [ ]:
messages_static = RBIC_static_messages(verbose=True, add_fewshot=False)

RB step 1: Certainly! Here's a brief explanation:

**Salary Range**: The salary range typically refers to the upper limit (minimum) and lower limit (maximum) of pay for the position.

**Currency**: This usually includes the currency used by the employer or company to calculate salaries. Common currencies include USD (United States Dollar), EUR (Euro), GBP (British Pound Sterling), CAD (Canadian Dollar), etc.

**Frequency**: Frequency might refer to how often the salary is paid out (e.g., monthly, quarterly). It could also indicate whether the salary is fixed or variable based on performance metrics, such as annual bonuses, commissions, etc.

For example:
- Minimum Salary Range: $50,000 - $70,000 USD
- Currency: USD
- Frequency: Monthly

If there's no clear information provided about salary ranges, currency, or frequencies, it would be best to provide more context or ask clarifying questions directly in the job description.

RB step 2: ['Here are some common phrases and numeric patterns

In [ ]:
test_pred_df = pd.DataFrame(columns=["y_pred"])

In [ ]:
for i in tqdm(range(len(test_df))):
    desc = {
        "job_title": test_df.iloc[i].job_title,
        "job_ad_details": test_df.iloc[i].job_ad_details,
        "nation_short_desc": test_df.iloc[i].nation_short_desc,
        "salary_additional_text": test_df.iloc[i].salary_additional_text,
    }
    desc_str = str(desc)

    label = test_df.iloc[i].y_true
    label_str = str(label)

    label_pred = RBIC(messages_static, desc_str)

    test_pred_df.loc[len(test_pred_df)] = label_pred

# export the dataframe to a new csv file
test_pred_df.to_csv('salary_labelled_test_set_qwen_rbic_preds.csv', index=False)

  0%|          | 0/567 [00:00<?, ?it/s]

step3 messages: 7
step4 messages: 9


  0%|          | 1/567 [00:05<48:40,  5.16s/it]

step5 messages: 10
Failed to parse model output as JSON:
step3 messages: 7
step4 messages: 9


  0%|          | 2/567 [00:12<58:06,  6.17s/it]

...

step5 messages: 10
Failed to parse model output as JSON:
step3 messages: 7
step4 messages: 9


100%|█████████▉| 566/567 [41:56<00:04,  4.04s/it]

step5 messages: 10
Failed to parse model output as JSON:
step3 messages: 7
step4 messages: 9


100%|██████████| 567/567 [41:58<00:00,  4.44s/it]

step5 messages: 10
Failed to parse model output as JSON:


In [ ]:
test_pred_df

,y_pred
0,"ERROR ```json\n{\n ""MinSalary"": """",\n ""MaxSa..."
1,"ERROR ```json\n{\n ""MinSalary"": ""50"",\n ""Max..."
2,"ERROR ```json\n{""MinSalary"": 50000, ""MaxSalary..."
3,"ERROR ```json\n{\n ""MinSalary"": """",\n ""MaxSa..."
4,"ERROR ```json\n{\n ""MinSalary"": ""50"",\n ""Max..."
...,...
562,"ERROR ```json\n{\n ""MinSalary"": """",\n ""M..."
563,"ERROR ```json\n{\n ""MinSalary"": ""$50,000"",\..."
564,"ERROR ```json\n{\n ""MinSalary"": """",\n ""MaxSa..."
565,"ERROR ```json\n{\n ""MinSalary"": """",\n ""MaxSa..."


In [ ]:
no_few_shot_df = pd.read_csv("salary_labelled_test_set_qwen_rbic_preds.csv")

,y_pred
0,"ERROR ```json\n{\n ""MinSalary"": """",\n ""MaxSa..."
1,"ERROR ```json\n{\n ""MinSalary"": ""50"",\n ""Max..."
2,"ERROR ```json\n{""MinSalary"": 50000, ""MaxSalary..."
3,"ERROR ```json\n{\n ""MinSalary"": """",\n ""MaxSa..."
4,"ERROR ```json\n{\n ""MinSalary"": ""50"",\n ""Max..."
...,...
562,"ERROR ```json\n{\n ""MinSalary"": """",\n ""M..."
563,"ERROR ```json\n{\n ""MinSalary"": ""$50,000"",\..."
564,"ERROR ```json\n{\n ""MinSalary"": """",\n ""MaxSa..."
565,"ERROR ```json\n{\n ""MinSalary"": """",\n ""MaxSa..."


### **Post processing no few shot**


In [ ]:
no_few_shot_df = pd.read_csv("salary_labelled_test_set_qwen_rbic_preds.csv")
no_few_shot_df

,y_pred
0,"ERROR ```json\n{\n ""MinSalary"": """",\n ""MaxSa..."
1,"ERROR ```json\n{\n ""MinSalary"": ""50"",\n ""Max..."
2,"ERROR ```json\n{""MinSalary"": 50000, ""MaxSalary..."
3,"ERROR ```json\n{\n ""MinSalary"": """",\n ""MaxSa..."
4,"ERROR ```json\n{\n ""MinSalary"": ""50"",\n ""Max..."
...,...
562,"ERROR ```json\n{\n ""MinSalary"": """",\n ""M..."
563,"ERROR ```json\n{\n ""MinSalary"": ""$50,000"",\..."
564,"ERROR ```json\n{\n ""MinSalary"": """",\n ""MaxSa..."
565,"ERROR ```json\n{\n ""MinSalary"": """",\n ""MaxSa..."


In [ ]:
post_process_df = pd.DataFrame(columns=["y_pred"])

In [ ]:
post_process_df = no_few_shot_df

In [ ]:
for i in no_few_shot_df.index:
  prediction = no_few_shot_df.iloc[i].y_pred
  if "ERROR" in no_few_shot_df.iloc[i].y_pred :
    cleaned_text = prediction.replace("ERROR", "").replace("```json", "").replace("```", "").strip()

    # remove any "}}" at the end
    cleaned_text = cleaned_text.replace("}}", "}")
    clue_json = cleaned_text[cleaned_text.find('{'):cleaned_text.find('}') + 1]
    if "Clue" in cleaned_text and "MinSalary" in cleaned_text and "MinSalary" not in clue_json:
      escaped_clue_json = re.escape(clue_json)
      cleaned_text = re.sub(escaped_clue_json + r',?\s*', '', cleaned_text)
    try:
      if "Clue" in cleaned_text:
        loaded = json.loads(cleaned_text)
        label = f"{loaded['Clue']}"
      else:
        loaded = json.loads(cleaned_text)
        label = f"{loaded['MinSalary']}-{loaded['MaxSalary']}-{loaded['Currency'].upper()}-{loaded['Frequency'].upper()}"
      post_process_df.loc[i, 'y_pred'] = label
    except:
      # not a json element or valid output tag
      post_process_df .loc[i, 'y_pred'] = cleaned_text


# post_process_df.to_csv('2_post_processed_salary_labelled_test_set_qwen_rbic_preds.csv', index=False)



In [ ]:
post_process_df

,y_pred
0,--USD-MONTHLY
1,50-70-USD-MONTHLY
2,50000-70000-USD-MONTHLY
3,--USD-MONTHLY
4,50-70-USD-MONTHLY
...,...
562,--USD-MONTHLY
563,"$50,000-$70,000-USD-MONTHLY"
564,--USD-MONTHLY
565,--USD-MONTHLY


In [ ]:
post_process_df.to_csv('2_post_processed_salary_labelled_test_set_qwen_rbic_preds.csv', index=False)

# Regular Prompting
Comparsion of RBIC to Prompting results


In [ ]:
test_pred_df = pd.DataFrame(columns=["y_pred"])

messages_static = [
    {"role": "system", "content": "You are an expert job ad annotator. Your task is to extract structured salary information from job descriptions in the format: min-max-currency-frequency. If salary is not found, return: 0-0-None-None."},
]

In [ ]:
for i in tqdm(range(len(test_df))):
    desc = {
        "job_title": test_df.iloc[i].job_title,
        "job_ad_details": test_df.iloc[i].job_ad_details,
        "nation_short_desc": test_df.iloc[i].nation_short_desc,
        "salary_additional_text": test_df.iloc[i].salary_additional_text,
    }
    desc_str = str(desc)

    messages = copy.deepcopy(messages_static)
    messages.append({
        "role": "user",
        "content": (
            f"{desc_str} Extract structured salary information from this job descriptions in the format: min-max-currency-frequency. "
            "Respond in JSON: {\"MinSalary\": \"\", \"MaxSalary\": \"\", \"Currency\": \"\", \"Frequency\": \"\"}. "
            "If not provided explicitly, output 0 for \"MinSalary\" and \"MaxSalary\", and \"None\" for \"Currency\" and \"Frequency\". "
            "If the salary is mentioned, always output a range, where MinSalary and MaxSalary can be equal. "
            "Use 'nation_short_desc' to determine the correct currency. "
            "Output the currency as 3 letters. Use adverb to output frequency (annual, monthly, daily or hourly)."
        )
    })

    response = call_model(messages)
    answer_str = response[0]

    # format the output
    try:
        answer_str_ = answer_str[answer_str.find('{'):answer_str.find('}') + 1]
        answer_str_ = answer_str_.replace('“', '"')
        answer_str_ = answer_str_.replace('”', '"')
        answer = json.loads(answer_str_)

        label = f"{answer['MinSalary']}-{answer['MaxSalary']}-{answer['Currency'].upper()}-{answer['Frequency'].upper()}"

    except json.JSONDecodeError:
        print(f"Failed to parse model output as JSON: {answer_str}")
        label = "ERROR " + answer_str

    test_pred_df.loc[len(test_pred_df)] = label

# export the dataframe to a new csv file
test_pred_df.to_csv('salary_labelled_test_set_qwen_preds.csv', index=False)

  5%|▌         | 29/567 [00:42<11:15,  1.25s/it]

Failed to parse model output as JSON: {"MinSalary": 1500, "MaxSalary": None, "Currency": "RM", "Frequency": "annual"}


 15%|█▌        | 86/567 [02:00<10:57,  1.37s/it]

Failed to parse model output as JSON: {"MinSalary": 21000, "MaxSalary": None, "Currency": "PHP", "Frequency": "Annual"}


 21%|██        | 120/567 [02:52<09:19,  1.25s/it]

...

 91%|█████████ | 516/567 [12:36<01:07,  1.33s/it]

Failed to parse model output as JSON: {"MinSalary": 75.00, "MaxSalary": None, "Currency": "HKD", "Frequency": "monthly"}


 91%|█████████▏| 518/567 [12:40<01:29,  1.82s/it]

Failed to parse model output as JSON: {"MinSalary": "RM\xa06,000 - RM\xa07,000", "MaxSalary": "RM\xa06,000 - RM\xa07,000", "Currency": "RM", "Frequency": "monthly"}


100%|██████████| 567/567 [13:52<00:00,  1.47s/it]


In [ ]:
for i in tqdm(range(len(test_df))):
    desc = {
        "job_title": test_df.iloc[i].job_title,
        "job_ad_details": test_df.iloc[i].job_ad_details,
        "nation_short_desc": test_df.iloc[i].nation_short_desc,
        "salary_additional_text": test_df.iloc[i].salary_additional_text,
    }
    desc_str = str(desc)

    messages = copy.deepcopy(messages_static)
    messages.append({
        "role": "user",
        "content": (
            f"{desc_str} Extract structured salary information from this job descriptions in the format: min-max-currency-frequency. "
            "Respond in JSON: {\"MinSalary\": \"\", \"MaxSalary\": \"\", \"Currency\": \"\", \"Frequency\": \"\"}. "
            "If not provided explicitly, output 0 for \"MinSalary\" and \"MaxSalary\", and \"None\" for \"Currency\" and \"Frequency\". "
            "If the salary is mentioned, always output a range, where MinSalary and MaxSalary can be equal. "
            "Use 'nation_short_desc' to determine the correct currency. "
            "Output the currency as 3 letters. Use adverb to output frequency (annual, monthly, daily or hourly)."
        )
    })

    response = call_model(messages)
    answer_str = response[0]

    # format the output
    try:
        answer_str_ = answer_str[answer_str.find('{'):answer_str.find('}') + 1]
        answer_str_ = answer_str_.replace('“', '"')
        answer_str_ = answer_str_.replace('”', '"')
        answer = json.loads(answer_str_)

        label = f"{answer['MinSalary']}-{answer['MaxSalary']}-{answer['Currency'].upper()}-{answer['Frequency'].upper()}"

    except json.JSONDecodeError:
        print(f"Failed to parse model output as JSON: {answer_str}")
        label = "ERROR " + answer_str

    test_pred_df.loc[len(test_pred_df)] = label

# export the dataframe to a new csv file
test_pred_df.to_csv('salary_labelled_test_set_qwen_preds.csv', index=False)

  5%|▌         | 29/567 [00:46<11:35,  1.29s/it]

Failed to parse model output as JSON: {"MinSalary": 1500, "MaxSalary": None, "Currency": "RM", "Frequency": "Annual"}


  8%|▊         | 44/567 [01:06<11:42,  1.34s/it]

Failed to parse model output as JSON: {"MinSalary": 25.05, "MaxSalary": None, "Currency": "AUD", "Frequency": "Annual"}


 10%|█         | 57/567 [01:24<11:18,  1.33s/it]

...

 89%|████████▉ | 506/567 [12:18<01:30,  1.49s/it]

Failed to parse model output as JSON: To provide accurate salary information, I need to analyze the job description more thoroughly. Please provide the full text of the job description so I can extract the required structured data accurately.


 91%|█████████ | 516/567 [12:32<01:09,  1.37s/it]

Failed to parse model output as JSON: {"MinSalary": 75.00, "MaxSalary": None, "Currency": "HKD", "Frequency": "annual"}


100%|██████████| 567/567 [13:47<00:00,  1.46s/it]


# Prompt Results

In [ ]:
salary_true = pd.read_csv("salary_labelled_test_set_cleaned.csv")

In [ ]:
salary_true.y_true

,y_true
0,1500-1800-MYR-MONTHLY
1,60-60-HKD-HOURLY
2,0-0-None-None
3,0-0-None-None
4,0-0-None-None
...,...
562,26-26-NZD-HOURLY
563,0-0-None-None
564,0-0-None-None
565,1500-2500-MYR-MONTHLY


In [ ]:
prompt = pd.read_csv("salary_labelled_test_set_qwen_preds.csv")

In [ ]:
prompt

,y_pred
0,0-1800-RM-MONTHLY
1,60-60-HKD-ANNUAL
2,60000-90000-AUD-ANNUAL
3,140000-210000-AUD-MONTHLY
4,25000-40000-USD-ANNUAL
...,...
562,26.24-None-NONE-MONTHLY
563,40000-70000-USD-ANNUAL
564,0-90000-AUD-ANNUAL
565,1500.0-2500.0-RM-MONTHLY


In [ ]:
post_process_prompt = prompt

In [ ]:
for i in prompt.index:
  prediction = prompt.iloc[i].y_pred

  if isinstance(prediction, str) and "ERROR" in prediction:
    cleaned_text = prediction.replace("ERROR", "").strip()
    # remove None to Null, otherwise cant json loads
    cleaned_text = re.sub(r'\bNone\b', 'null', cleaned_text)
    try:
      loaded = json.loads(cleaned_text)
      label = f"{loaded['MinSalary']}-{loaded['MaxSalary']}-{loaded['Currency'].upper()}-{loaded['Frequency'].upper()}"
    except:
      label = None
    post_process_prompt.loc[i, 'y_pred'] = label


In [ ]:
post_process_prompt

,y_pred
0,0-1800-RM-MONTHLY
1,60-60-HKD-ANNUAL
2,60000-90000-AUD-ANNUAL
3,140000-210000-AUD-MONTHLY
4,25000-40000-USD-ANNUAL
...,...
562,26.24-None-NONE-MONTHLY
563,40000-70000-USD-ANNUAL
564,0-90000-AUD-ANNUAL
565,1500.0-2500.0-RM-MONTHLY


Overall results

In [ ]:
total = len(post_process_prompt)
excat_match = (post_process_prompt["y_pred"] == salary_true["y_true"]).sum()
overall = (excat_match/ total) * 100
print(f"overall {overall:.2f}%")

overall 1.94%


accuray metrics

In [ ]:
df1_split = post_process_prompt['y_pred'].str.split('-', expand=True)
df1_split = df1_split.iloc[:, :4]
df2_split = salary_true['y_true'].str.split('-', expand=True)
df2_split = df2_split.iloc[:, :4]

df1_split.columns = ['MinSalary', 'MaxSalary', 'Currency', 'Frequency']
df2_split.columns = ['MinSalary', 'MaxSalary', 'Currency', 'Frequency']

min_salary_accuracy = (df1_split['MinSalary'] == df2_split['MinSalary']).mean()
max_salary_accuracy = (df1_split['MaxSalary'] == df2_split['MaxSalary']).mean()
currency_accuracy = (df1_split['Currency'] == df2_split['Currency']).mean()
frequency_accuracy = (df1_split['Frequency'] == df2_split['Frequency']).mean()


print(f"MinSalary Accuracy: {min_salary_accuracy * 100:.2f}%")
print(f"MaxSalary Accuracy: {max_salary_accuracy * 100:.2f}%")
print(f"Currency Accuracy: {currency_accuracy * 100:.2f}%")
print(f"Frequency Accuracy: {frequency_accuracy * 100:.2f}%")

MinSalary Accuracy: 27.87%
MaxSalary Accuracy: 19.40%
Currency Accuracy: 19.22%
Frequency Accuracy: 19.40%


# RBIC results

In [ ]:
RBIC = pd.read_csv("2_post_processed_salary_labelled_test_set_qwen_rbic_preds.csv")

In [ ]:
RBIC

,y_pred
0,--USD-MONTHLY
1,50-70-USD-MONTHLY
2,50000-70000-USD-MONTHLY
3,--USD-MONTHLY
4,50-70-USD-MONTHLY
...,...
562,--USD-MONTHLY
563,"$50,000-$70,000-USD-MONTHLY"
564,--USD-MONTHLY
565,--USD-MONTHLY


overall results

In [ ]:
total = len(salary_true)
excat_match = (RBIC["y_pred"] == salary_true["y_true"]).sum()
overall = (excat_match/ total) * 100
print(f"overall {overall:.2f}%")

overall 0.00%


In [ ]:
df1_split = RBIC['y_pred'].str.split('-', expand=True)
df1_split = df1_split.iloc[:, :4]
df2_split = salary_true['y_true'].str.split('-', expand=True)
df2_split = df2_split.iloc[:, :4]

df1_split.columns = ['MinSalary', 'MaxSalary', 'Currency', 'Frequency']
df2_split.columns = ['MinSalary', 'MaxSalary', 'Currency', 'Frequency']

min_salary_accuracy = (df1_split['MinSalary'] == df2_split['MinSalary']).mean()
max_salary_accuracy = (df1_split['MaxSalary'] == df2_split['MaxSalary']).mean()
currency_accuracy = (df1_split['Currency'] == df2_split['Currency']).mean()
frequency_accuracy = (df1_split['Frequency'] == df2_split['Frequency']).mean()


print(f"MinSalary Accuracy: {min_salary_accuracy * 100:.2f}%")
print(f"MaxSalary Accuracy: {max_salary_accuracy * 100:.2f}%")
print(f"Currency Accuracy: {currency_accuracy * 100:.2f}%")
print(f"Frequency Accuracy: {frequency_accuracy * 100:.2f}%")

MinSalary Accuracy: 7.58%
MaxSalary Accuracy: 6.88%
Currency Accuracy: 0.71%
Frequency Accuracy: 22.93%


# RBIC with fewshot

In [ ]:
salary_true

,job_id,job_title,job_ad_details,nation_short_desc,salary_additional_text,y_true
0,72527377,Cashier,Bertanggungjawab sebagai cashierMengurus semua...,MY,"RM 1,500 – RM 1,800 per month",1500-1800-MYR-MONTHLY
1,73593343,學校派飯員,編號 : 35-24-0004254 LA 兼職\n日期 : 01/02/2024\n職位 ...,HK,NaN,60-60-HKD-HOURLY
2,51294698,Skid Steer Operator/ Truck Driver,Company: Bitu-mill specializes in road profil...,AUS,NaN,0-0-None-None
3,56209744,Business Development Manager,"We’re the P&N Group, one of Australia’s larges...",AUS,NaN,0-0-None-None
4,63825213,Workplace and Safety Officer,ResponsibilitesThis person will be working w...,SG,-,0-0-None-None
...,...,...,...,...,...,...
562,59723262,Trainee Rail Safety Officer - Tauranga,Company description:KiwiRail LtdJob descriptio...,NZ,NaN,26-26-NZD-HOURLY
563,76585795,Procurement Manager - Logistics Company,Our client is one of the top logistics players...,ID,NaN,0-0-None-None
564,51727455,Process Operators - Annies Fruit Bars - Mornin...,"Our Business Since the time of the hekenga, ou...",NZ,NaN,0-0-None-None
565,70947712,HONG LEONG BANK RECOVERY OFFICER / CALL CENTRE,***Contact person : En.Aaron 0129737175 ( what...,MY,NaN,1500-2500-MYR-MONTHLY


In [ ]:
RBIC_few = pd.read_csv("post_processed_salary_labelled_test_set_qwen_rbic_fewshot_preds.csv")

In [ ]:
RBIC_few[0:566]

,y_pred
0,--THB-MONTHLY
1,$1000-$2000-USD-MONTHLY
2,$3000-$5000-USD-YEARLY
3,"$11,000-$11,000-USD-MONTHLY"
4,"$2,000-$3,000-USD-MONTHLY"
...,...
561,$2000-$3000-USD-YEARLY
562,$2000-$3000-USD-MONTHLY
563,$40.27-$51.74-AUD-YEARLY
564,Not Provided-Not Provided-THB-MONTHLY


In [ ]:
 salary_true['y_true']

,y_true
0,1500-1800-MYR-MONTHLY
1,60-60-HKD-HOURLY
2,0-0-None-None
3,0-0-None-None
4,0-0-None-None
...,...
562,26-26-NZD-HOURLY
563,0-0-None-None
564,0-0-None-None
565,1500-2500-MYR-MONTHLY


In [ ]:
total = len(salary_true)
excat_match = (RBIC_few[0:567]["y_pred"] == salary_true["y_true"]).sum()
overall = (excat_match/ total) * 100
print(f"overall {overall:.2f}%")

overall 0.00%


In [ ]:
df1_split = RBIC_few[0:567]['y_pred'].str.split('-', expand=True)
df1_split = df1_split.iloc[:, :4]
df2_split = salary_true['y_true'].str.split('-', expand=True)
df2_split = df2_split.iloc[:, :4]

df1_split.columns = ['MinSalary', 'MaxSalary', 'Currency', 'Frequency']
df2_split.columns = ['MinSalary', 'MaxSalary', 'Currency', 'Frequency']

min_salary_accuracy = (df1_split['MinSalary'] == df2_split['MinSalary']).mean()
max_salary_accuracy = (df1_split['MaxSalary'] == df2_split['MaxSalary']).mean()
currency_accuracy = (df1_split['Currency'] == df2_split['Currency']).mean()
frequency_accuracy = (df1_split['Frequency'] == df2_split['Frequency']).mean()


print(f"MinSalary Accuracy: {min_salary_accuracy * 100:.2f}%")
print(f"MaxSalary Accuracy: {max_salary_accuracy * 100:.2f}%")
print(f"Currency Accuracy: {currency_accuracy * 100:.2f}%")
print(f"Frequency Accuracy: {frequency_accuracy * 100:.2f}%")

MinSalary Accuracy: 3.00%
MaxSalary Accuracy: 1.23%
Currency Accuracy: 1.06%
Frequency Accuracy: 17.81%
